# Step3 - Finetuning

In the final fine-tuning step (Step 3), our attention shifted to the instance level, where the LLM was taught to interpret SHAP values accurately and respond to follow-up queries based on these values. For this purpose, we prepared a structured JSON document following the alpaca-format and employed a supervised learning approach to impart this specialized knowledge to the LLM. The following notebook describes how this supervised finetuning step was conducted.

## Start Training
To start the training, we have to restart the textgeneration-webui, using the following commands:

    conda activate textgen
    cd /home/[...]/text-generation-webui/
    python server.py --share --model models/Step2-model --load-in-8bit

Thereby, we load the selected base model (Step2-model) in 8bit as recommended by the text-generation-webui-documentation.

After the webui started, the following steps have been executed within the webui:

1. Switch to tab "Training"
2. Give the LoRA-file a new name - in our case "Step3_adapter"
3. Adapt the Hyperparameters for the training:
- Training-epochs: 20
- Learning-rate: 3e^-4
- LoRA Rank: 8
- LoRA Alpha: 16
- Batch Size: 128
- Micro Batch Size: 4
- Cutoff Length: 256
- LR Scheduler: linear
- Overlap Length: 128
- Prefer Newline Cut Length: 128
4. Go to the Tab "Formatted Dataset"
5. Adapt the Dataselection:
- Data-Format: alpaca-format
- Dataset: Step3_Training_data
- Evaluation Dataset: None
6. Click "Start LoRA Training"

The new LoRA-adapter was saved to the folder "text-generation-webui/loras/Step3_adapter".

## Create battery model
We decided to create the final battery model by merging the newly trained LORA-adapter (Step3_adapter) with the corresponding base model (Step2-model). Therefore, we executed the following code from this notebook:

### Load Base-model, tokenizer, LoRA-adapter

In [ ]:
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_path = "/home/jupyter-edt_wise34_tf2/text-generation-webui/models/Step2-model/"


model = AutoModelForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="offload",
    trust_remote_code=True
)
model.config.use_cache = False

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### Combine Base-model with LoRA-adapter

In [ ]:
peft_model = PeftModel.from_pretrained(model, "/home/jupyter-edt_wise34_tf2/text-generation-webui/loras/Step3_adapter/", is_trainable=True)

### Merge Base-model with LoRA-adapter 

In [5]:
merged_model = peft_model.merge_and_unload()

### Save merged model to text-generation-webui

In [ ]:
merged_model.save_pretrained("/home/jupyter-edt_wise34_tf2/text-generation-webui/models/battery_model/", safe_serializetion=True)
tokenizer.save_pretrained("/home/jupyter-edt_wise34_tf2/text-generation-webui/models/battery_model/")

The final battery model is saved to the folder "/home/[...]/text-generation-webui/models".